In [1]:
import pandas as pd
from minio import Minio
from minio.error import S3Error
import json, logging, os, glob
import json
import logging
import time
from typing import List
import pandas as pd
import polars as pl
import pyarrow as pa
import pyarrow.parquet as pq
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, col
from pyspark.sql.types import StringType, IntegerType, StructType, StructField, BooleanType, LongType, ArrayType
from pyspark.sql import functions as F, types as T


pd.options.mode.chained_assignment = None
bucket_name = 'iceberg-data'

# Insert data into Iceberg

In [2]:
client = Minio(
            "172.28.0.3:9000",  # MinIO server URL (e.g., localhost:9000)
            access_key="datdao",   # MinIO access key
            secret_key="tiendat123",   # MinIO secret key
            secure=False                    # Set to True if using HTTPS
        )

# List all buckets (this doesn't return region directly, but checks if connected properly)
object_list = []
objects = client.list_objects(bucket_name, prefix='data/mpd', recursive=True)
for obj in objects:
    object_list.append(obj.object_name)

In [17]:
conf = (
    SparkConf()
        .setAppName('MinIO Read Example')
        # .set("spark.sql.shuffle.partitions", "400")
        # .set("spark.memory.storageFraction", "0.7")
)
# Initialize Spark session
spark = SparkSession.builder.config(conf=conf)\
                            .enableHiveSupport()\
                            .getOrCreate()

# Specify the S3 path (MinIO path)
file_path = "s3a://iceberg-data/"

24/11/15 10:00:19 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [ ]:
spark.read

# Get file from S3 and insert to iceberg

In [ ]:
conf = (
    SparkConf()
        .setAppName('MinIO Read Example')
        # .set("spark.sql.shuffle.partitions", "400")
        # .set("spark.memory.storageFraction", "0.7")
)
# Initialize Spark session
spark = SparkSession.builder.config(conf=conf)\
                            .enableHiveSupport()\
                            .getOrCreate()

# Specify the S3 path (MinIO path)
file_path = "s3a://iceberg-data/"

schema = StructType([
    StructField("playlists", ArrayType(
        StructType([
            StructField("name", StringType(), True),
            StructField("collaborative", BooleanType(), True),
            StructField("pid", LongType(), True),
            StructField("modified_at", LongType(), True),
            StructField("num_tracks", LongType(), True),
            StructField("num_albums", LongType(), True),
            StructField("num_followers", LongType(), True),
            StructField("tracks", ArrayType(
                StructType([
                    StructField("pos", LongType(), True),
                    StructField("artist_name", StringType(), True),
                    StructField("track_uri", StringType(), True),
                    StructField("artist_uri", StringType(), True),
                    StructField("track_name", StringType(), True),
                    StructField("album_uri", StringType(), True),
                    StructField("duration_ms", LongType(), True),
                    StructField("album_name", StringType(), True)
                ])
            ), True),
            StructField("num_edits", LongType(), True),
            StructField("duration_ms", LongType(), True),
            StructField("num_artists", LongType(), True)
        ])
    ), True)
])

for index, value in enumerate(object_list):
    df_spark = spark.read\
                .option("multiLine", True) \
                .option("header",True) \
                .json(file_path + value, schema)

    df_playlists = df_spark.select(explode('playlists').alias('playlists')).select('playlists.*')

    df_playlists_exploded = df_playlists.withColumn('tracks_exploded', explode('tracks'))\
                                        .withColumnRenamed('duration_ms', 'duration_total_ms')\
                                        .select('*', 'tracks_exploded.*')\
                                        .drop("tracks", "tracks_exploded")
    if index == 0:
        df_playlists_exploded.writeTo('db.spotify_playlist').createOrReplace()
    else:
        df_playlists_exploded.writeTo('db.spotify_playlist').append()

24/11/15 10:00:53 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.
24/11/15 10:00:54 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


AnalysisException: [PATH_NOT_FOUND] Path does not exist: s3a://iceberg-data/opt/spark/data/mpd.slice.41000-41999.json.

# Get file from local and insert into iceberg

In [ ]:
conf = (
    SparkConf()
        .setAppName('MinIO Read Example')
        # .set("spark.sql.shuffle.partitions", "400")
        # .set("spark.memory.storageFraction", "0.7")
)
# Initialize Spark session
spark = SparkSession.builder.config(conf=conf)\
                            .enableHiveSupport()\
                            .getOrCreate()

# Specify the S3 path (MinIO path)
file_path = "s3a://iceberg-data/"

schema = StructType([
    StructField("playlists", ArrayType(
        StructType([
            StructField("name", StringType(), True),
            StructField("collaborative", BooleanType(), True),
            StructField("pid", LongType(), True),
            StructField("modified_at", LongType(), True),
            StructField("num_tracks", LongType(), True),
            StructField("num_albums", LongType(), True),
            StructField("num_followers", LongType(), True),
            StructField("tracks", ArrayType(
                StructType([
                    StructField("pos", LongType(), True),
                    StructField("artist_name", StringType(), True),
                    StructField("track_uri", StringType(), True),
                    StructField("artist_uri", StringType(), True),
                    StructField("track_name", StringType(), True),
                    StructField("album_uri", StringType(), True),
                    StructField("duration_ms", LongType(), True),
                    StructField("album_name", StringType(), True)
                ])
            ), True),
            StructField("num_edits", LongType(), True),
            StructField("duration_ms", LongType(), True),
            StructField("num_artists", LongType(), True)
        ])
    ), True)
])

for index, file in enumerate(glob.glob('/opt/spark/data/*')):
    df_spark = spark.read\
                .option("multiLine", True) \
                .option("header",True) \
                .json(file, schema)

    df_playlists = df_spark.select(explode('playlists').alias('playlists')).select('playlists.*')

    df_playlists_exploded = df_playlists.withColumn('tracks_exploded', explode('tracks'))\
                                        .withColumnRenamed('duration_ms', 'duration_total_ms')\
                                        .select('*', 'tracks_exploded.*')\
                                        .drop("tracks", "tracks_exploded")
    if index == 0:
        df_playlists_exploded.writeTo('db.spotify_playlist').createOrReplace()
    else:
        df_playlists_exploded.writeTo('db.spotify_playlist').append()

24/11/18 15:40:52 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
24/11/18 15:40:52 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
24/11/18 15:40:57 WARN ObjectStore: Version information not found in metastore. hive.metastore.schema.verification is not enabled so recording the schema version 2.3.0
24/11/18 15:40:57 WARN ObjectStore: setMetaStoreSchemaVersion called but recording version is disabled: version = 2.3.0, comment = Set by MetaStore UNKNOWN@172.23.0.4
24/11/18 15:40:57 WARN ObjectStore: Failed to get database default, returning NoSuchObjectException
24/11/18 15:40:58 WARN ObjectStore: Failed to get database global_temp, returning NoSuchObjectException
24/11/18 15:43:36 WARN S3FileIO: Unclosed S3FileIO instance created by:         
	org.apache.iceberg.aws.s3.S3FileIO.initialize(S3FileIO.java:359)
	org.apache.iceberg.CatalogUtil.loadFileIO(CatalogUtil.java:350)
	org.apache.iceberg.jdbc.JdbcCatalog.initialize(JdbcCatalog.java:130)
	

KeyboardInterrupt: 

24/11/18 15:46:50 ERROR TaskSchedulerImpl: Lost executor 0 on 172.23.0.7: worker lost: 172.23.0.7:38637 got disassociated
24/11/18 15:46:50 ERROR TaskSchedulerImpl: Lost executor 1 on 172.23.0.6: worker lost: 172.23.0.6:41429 got disassociated


In [5]:
spark.stop()

# Interact with iceberg

In [ ]:
spark = SparkSession.builder.appName("Jupyter_Iceberg").getOrCreate()

24/11/18 14:54:38 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [3]:
spark

In [ ]:
spark.sql('SELECT * FROM db.spotify_playlist').show()

24/11/18 14:54:42 WARN JdbcCatalog: JDBC catalog is initialized without view support. To auto-migrate the database's schema and enable view support, set jdbc.schema-version=V1


UnsupportedOperationException: JDBC catalog is initialized without view support. To auto-migrate the database's schema and enable view support, set jdbc.schema-version=V1

24/11/18 14:57:22 ERROR TaskSchedulerImpl: Lost executor 0 on 172.20.0.6: worker lost: 172.20.0.6:36455 got disassociated


In [ ]:
df = spark.sql('SELECT * FROM db.spotify_playlist')

24/11/15 10:17:26 WARN JdbcCatalog: JDBC catalog is initialized without view support. To auto-migrate the database's schema and enable view support, set jdbc.schema-version=V1


In [ ]:
col_playlist = 'name|collaborative|pid|modified_at|num_tracks|num_albums|num_followers|num_edits|duration_total_ms|num_artists'.split('|')

In [ ]:
df.select(col_playlist).distinct().writeTo('db.playlist').createOrReplace()

In [ ]:
df.select('track_uri', 'track_name', 'duration_ms', 'artist_uri', 'album_uri').writeTo('db.track').createOrReplace()

In [ ]:
df.select('artist_uri', 'artist_name').distinct().writeTo('db.artist').createOrReplace()

In [ ]:
df.select('album_uri', 'album_name').distinct().writeTo('db.album').createOrReplace()

In [ ]:
df.select('album_uri', 'album_name').distinct().writeTo('db.album').createOrReplace()

In [ ]:
df.withColumn('playlist_track_id', F.concat('pid', 'track_uri')).select('playlist_track_id', 'pos', 'pid', 'track_uri').writeTo('db.playlist_track').createOrReplace()

In [ ]:
df.show()

+-----------+-------------+------+-----------+----------+----------+-------------+---------+-----------------+-----------+---+--------------------+--------------------+--------------------+-----------------------------+--------------------+-----------+-----------------------------+
|       name|collaborative|   pid|modified_at|num_tracks|num_albums|num_followers|num_edits|duration_total_ms|num_artists|pos|         artist_name|           track_uri|          artist_uri|                   track_name|           album_uri|duration_ms|                   album_name|
+-----------+-------------+------+-----------+----------+----------+-------------+---------+-----------------+-----------+---+--------------------+--------------------+--------------------+-----------------------------+--------------------+-----------+-----------------------------+
|Anime Songs|         NULL|166000| 1474156800|         6|         6|            1|        3|          1292341|          3|  0|   Raon Lee & PelleK|spot

In [34]:
spark.sql('SHOW TABLES IN db').show()

+---------+----------------+-----------+
|namespace|       tableName|isTemporary|
+---------+----------------+-----------+
|       db|        playlist|      false|
|       db|           track|      false|
|       db|          artist|      false|
|       db|           album|      false|
|       db|  playlist_track|      false|
|       db|spotify_playlist|      false|
+---------+----------------+-----------+



In [27]:
spark.sql('SELECT * FROM db.table').show()

UnsupportedOperationException: JDBC catalog is initialized without view support. To auto-migrate the database's schema and enable view support, set jdbc.schema-version=V1

In [25]:
def run():

    client = Minio(
                "192.168.160.3:9000",  # MinIO server URL (e.g., localhost:9000)
                access_key="datdao",   # MinIO access key
                secret_key="tiendat123",   # MinIO secret key
                secure=False                    # Set to True if using HTTPS
            )
    object_list = []
    objects = client.list_objects(bucket_name, recursive=True)
    for obj in objects:
        object_list.append(obj.object_name)
    return client.fget_object(bucket_name, object_list[-1], './json/' + object_list[-1].split('/')[-1])

# object_list = []
# objects = client.list_objects(bucket_name, prefix='mpd.', recursive=True)
    

In [ ]:


def run(spark):
    object_list = []
    objects = client.list_objects(bucket_name, recursive=True)
    for obj in objects:
        object_list.append(obj.object_name)
    for x in object_list:
        client.fget_object(bucket_name, x, './json/test.json')
        
        def LstJson():
            path = '/home/iceberg/notebooks/json'
            filenames = os.listdir(path)
            lst = []
            for filename in sorted(filenames):
                if filename.startswith("test") and filename.endswith(".json"):
                    fullpath = os.sep.join((path, filename))
                    f = open(fullpath)
                    js = f.read()
                    f.close()
                    mpd_slice = json.loads(js)
                    for playlist in mpd_slice["playlists"]:
                        lst.append(playlist)
            return lst, playlist.keys()

        sub_schema = StructType([
            StructField('pos', IntegerType(), True),
            StructField('artist_name', StringType(), True),
            StructField('artist_uri', StringType(), True),
            StructField('album_name', StringType(), True),
            StructField('track_uri', StringType(), True),
            StructField('album_uri', StringType(), True),
            StructField('track_name', StringType(), True),
            StructField('duration_ms', IntegerType(), True),
        ])

        schema = StructType([
            StructField('name', StringType(), True),
            StructField('pid', IntegerType(), True),
            StructField('num_albums', IntegerType(), True),
            StructField('num_artists', IntegerType(), True),
            StructField('num_edits', IntegerType(), True),
            StructField('num_followers', IntegerType(), True),
            StructField('duration_ms', IntegerType(), True),
            StructField('modified_at', IntegerType(), True),
            StructField('collaborative', StringType(), True),
            StructField('description', StringType(), True),
            StructField('tracks', ArrayType(sub_schema), True)
        ])
        df_sp = spark.createDataFrame(LstJson()[0],schema)
        df_unnested = df_sp.withColumn('tracks', explode('tracks')).select('*', "tracks.*")
        df_unnested.writeTo("db.spotify_playlist").append()


ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/opt/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving


KeyboardInterrupt: 

In [22]:
def LstJson():
    path = '/home/iceberg/notebooks/json'
    filenames = os.listdir(path)
    lst = []
    for filename in sorted(filenames):
        if filename.startswith("test") and filename.endswith(".json"):
            fullpath = os.sep.join((path, filename))
            f = open(fullpath)
            js = f.read()
            f.close()
            mpd_slice = json.loads(js)
            for playlist in mpd_slice["playlists"]:
                lst.append(playlist)
    return lst, playlist.keys()

sub_schema = StructType([
    StructField('pos', IntegerType(), True),
    StructField('artist_name', StringType(), True),
    StructField('artist_uri', StringType(), True),
    StructField('album_name', StringType(), True),
    StructField('track_uri', StringType(), True),
    StructField('album_uri', StringType(), True),
    StructField('track_name', StringType(), True),
    StructField('duration_ms', IntegerType(), True),
])

schema = StructType([
    StructField('name', StringType(), True),
    StructField('pid', IntegerType(), True),
    StructField('num_albums', IntegerType(), True),
    StructField('num_artists', IntegerType(), True),
    StructField('num_edits', IntegerType(), True),
    StructField('num_followers', IntegerType(), True),
    StructField('duration_ms', IntegerType(), True),
    StructField('modified_at', IntegerType(), True),
    StructField('collaborative', StringType(), True),
    StructField('description', StringType(), True),
    StructField('tracks', ArrayType(sub_schema), True)
])
df_sp = spark.createDataFrame(LstJson()[0],schema)
df_unnested = df_sp.withColumnRenamed('duration_ms', 'duration_playlist_ms').withColumn('tracks', explode('tracks')).select('*', "tracks.*")

In [24]:
df_unnested.show()

24/10/28 14:57:05 WARN TaskSetManager: Stage 6 contains a task of very large size (1954 KiB). The maximum recommended task size is 1000 KiB.


+----------+---+----------+-----------+---------+-------------+--------------------+-----------+-------------+-----------+--------------------+---+------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------+
|      name|pid|num_albums|num_artists|num_edits|num_followers|duration_playlist_ms|modified_at|collaborative|description|              tracks|pos|       artist_name|          artist_uri|          album_name|           track_uri|           album_uri|          track_name|duration_ms|
+----------+---+----------+-----------+---------+-------------+--------------------+-----------+-------------+-----------+--------------------+---+------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------+
|Throwbacks|  0|        47|         37|        6|            1|            11532414| 1493424000|        false|       NULL|{0, Missy Elliott...|  0| 

In [251]:
df_unnested.writeTo("db.spotify_playlist").append()

24/10/23 11:26:58 WARN TaskSetManager: Stage 40 contains a task of very large size (1094 KiB). The maximum recommended task size is 1000 KiB.


In [2]:
def get_object_list(bucket_name: str, prefix: str) -> List[str]:
    '''
    Gets a list of objects from a bucket.
    '''
    logger = logging.getLogger('gsod_logger')
    logger.setLevel(logging.INFO)

    # # Load the credentials and connection information.
    # with open('credentials.json') as f:
    #     credentials = json.load(f)

    # Get data of an object.
    try:
        # MinIO client initialization
        client = Minio(
            "192.168.160.3:9000",  # MinIO server URL (e.g., localhost:9000)
            access_key="datdao",   # MinIO access key
            secret_key="tiendat123",   # MinIO secret key
            secure=False                    # Set to True if using HTTPS
        )

        object_list = []
        objects = client.list_objects(bucket_name, prefix=prefix, recursive=True)
        for obj in objects:
            object_list.append(obj.object_name)

    except S3Error as s3_err:
        logger.error(f'S3 Error occurred: {s3_err}.')
        raise s3_err
    except Exception as err:
        logger.error(f'Error occurred: {err}.')
        raise err

    return object_list

In [129]:
def process_playlists(path):
    filenames = os.listdir(path)
    for filename in sorted(filenames):
        if filename.startswith("mpd.slice.") and filename.endswith(".json"):
            fullpath = os.sep.join((path, filename))
            f = open(fullpath)
            js = f.read()
            f.close()
            mpd_slice = json.loads(js)
            for playlist in mpd_slice["playlists"]:
                print_playlist(playlist)

def print_playlist(playlist):
    print("=====", playlist["pid"], "====")
    print("name:          ", playlist["name"])
    ts = time.strftime(
        "%Y-%m-%d %H:%M:%S", time.localtime(playlist["modified_at"])
    )

    print("last_modified: ", ts)
    print("num edits: ", playlist["num_edits"])
    print("num followers: ", playlist["num_followers"])
    print("num artists: ", playlist["num_artists"])
    print("num albums: ", playlist["num_albums"])
    print("num tracks: ", playlist["num_tracks"])
    print()
    for i, track in enumerate(playlist["tracks"]):
        print(
            "   %3d %s - %s - %s"
            % (i + 1, track["track_name"], track["album_name"], track["artist_name"])
        )
    print()

In [4]:
def LstJson():
    path = '/home/iceberg/notebooks/json'
    filenames = os.listdir(path)
    lst = []
    for filename in sorted(filenames):
        if filename.startswith("test") and filename.endswith(".json"):
            fullpath = os.sep.join((path, filename))
            f = open(fullpath)
            js = f.read()
            f.close()
            mpd_slice = json.loads(js)
            for playlist in mpd_slice["playlists"]:
                lst.append(playlist)
    return lst, playlist.keys()

In [6]:
df = pl.DataFrame(LstJson()[0])

In [ ]:
spark.create

In [26]:
df_sp = spark.createDataFrame(LstJson()[0],schema)
df_sp.show()

24/10/28 14:58:03 WARN TaskSetManager: Stage 8 contains a task of very large size (1954 KiB). The maximum recommended task size is 1000 KiB.


+--------------------+---+----------+-----------+---------+-------------+-----------+-----------+-------------+-----------+--------------------+
|                name|pid|num_albums|num_artists|num_edits|num_followers|duration_ms|modified_at|collaborative|description|              tracks|
+--------------------+---+----------+-----------+---------+-------------+-----------+-----------+-------------+-----------+--------------------+
|          Throwbacks|  0|        47|         37|        6|            1|   11532414| 1493424000|        false|       NULL|[{0, Missy Elliot...|
|    Awesome Playlist|  1|        23|         21|        5|            1|   11656470| 1506556800|        false|       NULL|[{0, Survivor, sp...|
|             korean |  2|        51|         31|       18|            1|   14039958| 1505692800|        false|       NULL|[{0, Hoody, spoti...|
|                 mat|  3|       107|         86|        4|            1|   28926058| 1501027200|        false|       NULL|[{0, Ca

In [232]:
df_unnested.writeTo("db.spotify_playlist").createOrReplace()

24/10/23 10:31:10 WARN TaskSetManager: Stage 35 contains a task of very large size (1094 KiB). The maximum recommended task size is 1000 KiB.


In [234]:
spark.table('db.spotify_playlist').show()

+----------+---+----------+-----------+---------+-------------+-----------+-----------+-------------+-----------+--------------------+
|      name|pid|num_albums|num_artists|num_edits|num_followers|duration_ms|modified_at|collaborative|description|              tracks|
+----------+---+----------+-----------+---------+-------------+-----------+-----------+-------------+-----------+--------------------+
|Throwbacks|  0|        47|         37|        6|            1|   11532414| 1493424000|        false|       NULL|{0, Missy Elliott...|
|Throwbacks|  0|        47|         37|        6|            1|   11532414| 1493424000|        false|       NULL|{1, Britney Spear...|
|Throwbacks|  0|        47|         37|        6|            1|   11532414| 1493424000|        false|       NULL|{2, Beyoncé, spot...|
|Throwbacks|  0|        47|         37|        6|            1|   11532414| 1493424000|        false|       NULL|{3, Justin Timber...|
|Throwbacks|  0|        47|         37|        6|      

In [173]:
from pyspark.sql import functions as F

# Flatten the struct by aliasing the columns
df_unnested = df.select(
    F.col("tracks.index").alias("index"),
    F.col("tracks.artist").alias("artist"),
    F.col("tracks.track_uri").alias("track_uri"),
    F.col("tracks.artist_uri").alias("artist_uri"),
    F.col("tracks.track_name").alias("track_name"),
    F.col("tracks.album_uri").alias("album_uri"),
    F.col("tracks.duration_ms").alias("duration_ms"),
    F.col("tracks.album_name").alias("album_name")
)

# Show the result
df_unnested.show()

TypeError: 'Column' object is not callable

In [30]:
pddf = df[['pid', 'tracks']].to_pandas()

In [7]:
pddf.explode('tracks')['tracks']

NameError: name 'pddf' is not defined

In [9]:
df.lazy()\
    .select('pid', 'tracks') \
    .explode('tracks') \
    .unnest('tracks') \
    .with_columns(
        playlist_track_id = 'track_' + pl.col('pid').cast(pl.Utf8) + pl.col('pos').cast(pl.Utf8)
    ) \
    .collect()

pid,pos,artist_name,track_uri,artist_uri,track_name,album_uri,duration_ms,album_name,playlist_track_id
i64,i64,str,str,str,str,str,i64,str,str
0,0,"""Missy Elliott""","""spotify:track:0UaMYEvWZi0ZqiDO…","""spotify:artist:2wIVse2owClT7go…","""Lose Control (feat. Ciara & Fa…","""spotify:album:6vV5UrXcfyQD1wu4…",226863,"""The Cookbook""","""track_00"""
0,1,"""Britney Spears""","""spotify:track:6I9VzXrHxO9rA9A5…","""spotify:artist:26dSoYclwsYLMAK…","""Toxic""","""spotify:album:0z7pVBGOD7HCIB7S…",198800,"""In The Zone""","""track_01"""
0,2,"""Beyoncé""","""spotify:track:0WqIKmW4BTrj3eJF…","""spotify:artist:6vWDO969PvNqNYH…","""Crazy In Love""","""spotify:album:25hVFAxTlDvXbx2X…",235933,"""Dangerously In Love (Alben für…","""track_02"""
0,3,"""Justin Timberlake""","""spotify:track:1AWQoqb9bSvzTjaL…","""spotify:artist:31TPClRtHm23Ris…","""Rock Your Body""","""spotify:album:6QPkyl04rXwTGlGl…",267266,"""Justified""","""track_03"""
0,4,"""Shaggy""","""spotify:track:1lzr43nnXAijIGYn…","""spotify:artist:5EvFsr3kj42KNv9…","""It Wasn't Me""","""spotify:album:6NmFmPX56pcLBOFM…",227600,"""Hot Shot""","""track_04"""
…,…,…,…,…,…,…,…,…,…
999,39,"""James Arthur""","""spotify:track:5uCax9HTNlzGybIS…","""spotify:artist:4IWBUUAFIplrNta…","""Say You Won't Let Go""","""spotify:album:7oiJYvEJHsmYtrgv…",211466,"""Back from the Edge""","""track_99939"""
999,40,"""Big Words""","""spotify:track:0P1oO2gREMYUCoOk…","""spotify:artist:0sHN89qak07mnug…","""The Answer""","""spotify:album:5jrsRHRAmetu5e7R…",263679,"""Hollywood, a Beautiful Coincid…","""track_99940"""
999,41,"""Allan Rayman""","""spotify:track:2oM4BuruDnEvk59I…","""spotify:artist:6Yv6OBXD6ZQakEl…","""25.22""","""spotify:album:3CbNgBzI7r9o0F6V…",189213,"""Roadhouse 01""","""track_99941"""


In [42]:
df_explode = df[['pid', 'tracks']].explode('tracks').unnest('tracks')

In [46]:
keys = df_explode['tracks'][0].keys()
keys

dict_keys(['pos', 'artist_name', 'track_uri', 'artist_uri', 'track_name', 'album_uri', 'duration_ms', 'album_name'])

In [ ]:
df_explode.with_columns(
    for x in keys:
        pl.col("tags").str.json_path_match(r"$.name").alias("name")
)

In [52]:
df_lazy = df_explode.lazy()
for x in keys:
    df_lazy.with_columns(
        pl.col("tracks").str.json_path_match(x).alias(x)
    )

AttributeError: 'Expr' object has no attribute 'json_path_match'

In [51]:
df_lazy.collect()

pid,tracks
i64,struct[8]
0,"{0,""Missy Elliott"",""spotify:track:0UaMYEvWZi0ZqiDOoHU3YI"",""spotify:artist:2wIVse2owClT7go1WT98tk"",""Lose Control (feat. Ciara & Fat Man Scoop)"",""spotify:album:6vV5UrXcfyQD1wu4Qo2I9K"",226863,""The Cookbook""}"
0,"{1,""Britney Spears"",""spotify:track:6I9VzXrHxO9rA9A5euc8Ak"",""spotify:artist:26dSoYclwsYLMAKD3tpOr4"",""Toxic"",""spotify:album:0z7pVBGOD7HCIB7S8eLkLI"",198800,""In The Zone""}"
0,"{2,""Beyoncé"",""spotify:track:0WqIKmW4BTrj3eJFmnCKMv"",""spotify:artist:6vWDO969PvNqNYHIOW5v0m"",""Crazy In Love"",""spotify:album:25hVFAxTlDvXbx2X2QkUkE"",235933,""Dangerously In Love (Alben für die Ewigkeit)""}"
0,"{3,""Justin Timberlake"",""spotify:track:1AWQoqb9bSvzTjaLralEkT"",""spotify:artist:31TPClRtHm23RisEBtV3X7"",""Rock Your Body"",""spotify:album:6QPkyl04rXwTGlGlcYaRoW"",267266,""Justified""}"
0,"{4,""Shaggy"",""spotify:track:1lzr43nnXAijIGYnCT8M8H"",""spotify:artist:5EvFsr3kj42KNv97ZEnqij"",""It Wasn't Me"",""spotify:album:6NmFmPX56pcLBOFMhIiKvF"",227600,""Hot Shot""}"
…,…
999,"{39,""James Arthur"",""spotify:track:5uCax9HTNlzGybIStD3vDh"",""spotify:artist:4IWBUUAFIplrNtaOHcJPRM"",""Say You Won't Let Go"",""spotify:album:7oiJYvEJHsmYtrgviAVIBD"",211466,""Back from the Edge""}"
999,"{40,""Big Words"",""spotify:track:0P1oO2gREMYUCoOkzYAyFu"",""spotify:artist:0sHN89qak07mnug3LVVjzP"",""The Answer"",""spotify:album:5jrsRHRAmetu5e7RRBoxj7"",263679,""Hollywood, a Beautiful Coincidence""}"
999,"{41,""Allan Rayman"",""spotify:track:2oM4BuruDnEvk59IvIXCwn"",""spotify:artist:6Yv6OBXD6ZQakEljaGaDAk"",""25.22"",""spotify:album:3CbNgBzI7r9o0F6VjH9sTY"",189213,""Roadhouse 01""}"


In [41]:
['tracks'][0].keys()

dict_keys(['pos', 'artist_name', 'track_uri', 'artist_uri', 'track_name', 'album_uri', 'duration_ms', 'album_name'])

In [7]:
df_spark = spark.createDataFrame(df)

PySparkTypeError: [CANNOT_INFER_TYPE_FOR_FIELD] Unable to infer the type of the field `_s`.

In [10]:
df.unique(subset=['collaborative'])

name,collaborative,pid,modified_at,num_tracks,num_albums,num_followers,tracks,num_edits,duration_ms,num_artists,description
str,str,i64,i64,i64,i64,i64,list[struct[8]],i64,i64,i64,str
"""Dave""","""true""",98,1435881600,81,20,1,"[{0,""Dave Matthews Band"",""spotify:track:474HmRcCZuGFV7i0jMNfEL"",""spotify:artist:2TI7qyDE0QfyOlnbtfDo7L"",""The Song That Jane Likes"",""spotify:album:2m60nm1PHbHd0o5uM0VQco"",203253,""Remember Two Things""}, {1,""Dave Matthews Band"",""spotify:track:7sxabPG0y5BpVR9w72bMm8"",""spotify:artist:2TI7qyDE0QfyOlnbtfDo7L"",""Cry Freedom"",""spotify:album:3Z72KfamjH9Wc5m9mgVqI7"",352666,""Crash""}, … {80,""Mac Demarco"",""spotify:track:5bVFRVVFtgQEVALsHYw5BT"",""spotify:artist:3Sz7ZnJQBIHsXLUSo0OQtM"",""Still Together"",""spotify:album:0Skv3s5A99n7dstiJOs0aA"",219373,""2""}]",8,26901306,3,null
"""Throwbacks""","""false""",0,1493424000,52,47,1,"[{0,""Missy Elliott"",""spotify:track:0UaMYEvWZi0ZqiDOoHU3YI"",""spotify:artist:2wIVse2owClT7go1WT98tk"",""Lose Control (feat. Ciara & Fat Man Scoop)"",""spotify:album:6vV5UrXcfyQD1wu4Qo2I9K"",226863,""The Cookbook""}, {1,""Britney Spears"",""spotify:track:6I9VzXrHxO9rA9A5euc8Ak"",""spotify:artist:26dSoYclwsYLMAKD3tpOr4"",""Toxic"",""spotify:album:0z7pVBGOD7HCIB7S8eLkLI"",198800,""In The Zone""}, … {51,""Boys Like Girls"",""spotify:track:6GIrIt2M39wEGwjCQjGChX"",""spotify:artist:0vWCyXMrrvMlCcepuOJaGI"",""The Great Escape"",""spotify:album:4WqgusSAgXkrjbXzqdBY68"",206520,""Boys Like Girls""}]",6,11532414,37,null


In [35]:
df

name,collaborative,pid,modified_at,num_tracks,num_albums,num_followers,tracks,num_edits,duration_ms,num_artists,description
str,str,i64,i64,i64,i64,i64,list[struct[8]],i64,i64,i64,str
"""Throwbacks""","""false""",0,1493424000,52,47,1,"[{0,""Missy Elliott"",""spotify:track:0UaMYEvWZi0ZqiDOoHU3YI"",""spotify:artist:2wIVse2owClT7go1WT98tk"",""Lose Control (feat. Ciara & Fat Man Scoop)"",""spotify:album:6vV5UrXcfyQD1wu4Qo2I9K"",226863,""The Cookbook""}, {1,""Britney Spears"",""spotify:track:6I9VzXrHxO9rA9A5euc8Ak"",""spotify:artist:26dSoYclwsYLMAKD3tpOr4"",""Toxic"",""spotify:album:0z7pVBGOD7HCIB7S8eLkLI"",198800,""In The Zone""}, … {51,""Boys Like Girls"",""spotify:track:6GIrIt2M39wEGwjCQjGChX"",""spotify:artist:0vWCyXMrrvMlCcepuOJaGI"",""The Great Escape"",""spotify:album:4WqgusSAgXkrjbXzqdBY68"",206520,""Boys Like Girls""}]",6,11532414,37,null
"""Awesome Playlist""","""false""",1,1506556800,39,23,1,"[{0,""Survivor"",""spotify:track:2HHtWyy5CgaQbC7XSoOb0e"",""spotify:artist:26bcq2nyj5GB7uRr558iQg"",""Eye of the Tiger"",""spotify:album:4PT9VulQaQP6XR1xBI2x1W"",243773,""Eye Of The Tiger""}, {1,""Daniel Tidwell"",""spotify:track:1MYYt7h6amcrauCOoso3Gx"",""spotify:artist:7zdmbPudNX4SQJXnYIuCTC"",""Libera Me From Hell (Tengen Toppa Gurren Lagann)"",""spotify:album:3q8vR3PFV8kG1m1Iv8DpKq"",70294,""Versus Hollywood""}, … {38,""Rush"",""spotify:track:1k0GxoZYv3Yx5mNeXMOZN2"",""spotify:artist:2Hkut4rAAyrQxRdof7FVJq"",""Vital Signs"",""spotify:album:2xg7iIKoSqaDNpDbJnyCjY"",286226,""Moving Pictures""}]",5,11656470,21,null
"""korean ""","""false""",2,1505692800,64,51,1,"[{0,""Hoody"",""spotify:track:74tqql9zP6JjF5hjkHHUXp"",""spotify:artist:7lXgbtBDcCRbfc5f8FhGUL"",""Like You"",""spotify:album:7hq1c0ZrLiSKxzhoRacoG7"",212715,""On And On""}, {1,""Loco"",""spotify:track:4erhEGuOGQgjv3p1bccnpn"",""spotify:artist:2e4G04F77jxVuDYo44TCSm"",""GOOD (feat. ELO)"",""spotify:album:4UlGIso11ixt3WiNUPZ4kT"",180006,""GOOD (feat. ELO)""}, … {63,""BTS"",""spotify:track:2SYa5Lx1uoCvyDIW4oee9b"",""spotify:artist:3Nrfpe0tUJi4K4DXYWgMUX"",""MIC Drop"",""spotify:album:2FTS6a6DLXMNp8flyA0HGO"",238193,""Love Yourself 承 'Her'""}]",18,14039958,31,null
"""mat""","""false""",3,1501027200,126,107,1,"[{0,""Camille Saint-Saëns"",""spotify:track:4WJ7UMD4i6DOPzyXU5pZSz"",""spotify:artist:436sYg6CZhNefQJogaXeK0"",""Danse macabre"",""spotify:album:0T9YCy8TruLD6Z4qiCGSn6"",428560,""French Festival""}, {1,""Camille Saint-Saëns"",""spotify:track:1Kzxd1kkjaGX4JZz2CYsXB"",""spotify:artist:436sYg6CZhNefQJogaXeK0"",""Piano concerto No. 2 in G Minor, Op. 22: Piano concerto No. 2 in G Minor, Op. 22: II. Allegro scherzando"",""spotify:album:4GunE6Xrwkb93FgXOYoxkP"",352160,""Grieg, Saint-Saëns: Piano Concertos""}, … {125,""Mild High Club"",""spotify:track:37vOv8JYJRsqfPAUQIfpRh"",""spotify:artist:5J81VungUjSVHxlPpTI9KG"",""Elegy"",""spotify:album:2PACdBic0FKBURz7AXV6sF"",171506,""Timeline""}]",4,28926058,86,null
"""90s""","""false""",4,1401667200,17,16,2,"[{0,""The Smashing Pumpkins"",""spotify:track:4iCGSi1RonREsPtfEKYj5b"",""spotify:artist:40Yq4vzPs9VNUrIBG5Jr2i"",""Tonight, Tonight"",""spotify:album:4bPT6Q8ppaSNppk1kbEbLl"",254626,""Mellon Collie and the Infinite Sadness""}, {1,""Oasis"",""spotify:track:5qqabIl2vWzo9ApSC317sa"",""spotify:artist:2DaxqgrOhkeH0fpeiQq2f4"",""Wonderwall - Remastered"",""spotify:album:1VW1MFNstaJuygaoTPkdCk"",258773,""(What's The Story) Morning Glory? [Remastered]""}, … {16,""Hootie & The Blowfish"",""spotify:track:1OFKUn2VLafrHj7ybnap0Q"",""spotify:artist:08ct2eZF5lUPdJpHwNKWof"",""Only Wanna Be With You"",""spotify:album:5AYmpTfdv1OoASUJ5rZB7K"",229586,""Cracked Rear View""}]",7,4335282,16,null
…,…,…,…,…,…,…,…,…,…,…,…
"""Praise""","""false""",999995,1409529600,32,23,1,"[{0,""Unspoken"",""spotify:track:2AMSWGnT7NCvOAPXxuRxIh"",""spotify:artist:0CqDqmyIQ9VQHQp52OEbB0"",""Who You Are"",""spotify:album:3E8lDVGWQKO7BOxRmcsayi"",204013,""Unspoken""}, {1,""NEEDTOBREATHE"",""spotify:track:6iz58z2LC8uHL3H2aiwX6I"",""spotify:artist:610EjgFatGvVPtib97jQ8G"",""So

In [34]:
df.filter(pl.col('collaborative') == 'true')#.sort(by='num_tracks', descending=True)['tracks']

name,collaborative,pid,modified_at,num_tracks,num_albums,num_followers,tracks,num_edits,duration_ms,num_artists,description
str,str,i64,i64,i64,i64,i64,list[struct[8]],i64,i64,i64,str
"""Dave""","""true""",98,1435881600,81,20,1,"[{0,""Dave Matthews Band"",""spotify:track:474HmRcCZuGFV7i0jMNfEL"",""spotify:artist:2TI7qyDE0QfyOlnbtfDo7L"",""The Song That Jane Likes"",""spotify:album:2m60nm1PHbHd0o5uM0VQco"",203253,""Remember Two Things""}, {1,""Dave Matthews Band"",""spotify:track:7sxabPG0y5BpVR9w72bMm8"",""spotify:artist:2TI7qyDE0QfyOlnbtfDo7L"",""Cry Freedom"",""spotify:album:3Z72KfamjH9Wc5m9mgVqI7"",352666,""Crash""}, … {80,""Mac Demarco"",""spotify:track:5bVFRVVFtgQEVALsHYw5BT"",""spotify:artist:3Sz7ZnJQBIHsXLUSo0OQtM"",""Still Together"",""spotify:album:0Skv3s5A99n7dstiJOs0aA"",219373,""2""}]",8,26901306,3,null
"""Party Rock!!""","""true""",106,1497916800,114,105,1,"[{0,""Nadia Ali"",""spotify:track:3KT0wY2cGC8kDJMGDLx751"",""spotify:artist:1C60viSZv6BoYtrnkZ44g5"",""Pressure - Alesso Edit"",""spotify:album:2gNcZlQ0LtvLCmDwKCSsMp"",184960,""Pressure""}, {1,""Madonna"",""spotify:track:1oHClQEgDmmbcEx12Kc5nZ"",""spotify:artist:6tbjWDEIzxoDsBA1FuhfPW"",""4 Minutes (feat. Justin Timberlake & Timbaland) - feat. Justin Timberlake And Timbaland"",""spotify:album:43lok9zd7BW5CoYkXZs7S0"",189693,""Celebration""}, … {113,""Zedd"",""spotify:track:0dA2Mk56wEzDgegdC6R17g"",""spotify:artist:2qxJFvFYMEDqd7ui6kSAcq"",""Stay (with Alessia Cara)"",""spotify:album:0VMGOBhLrC5Q2bfZnrocVN"",210090,""Stay""}]",26,29550753,100,null
"""4/20""","""true""",112,1461110400,65,65,1,"[{0,""Peaches"",""spotify:track:43Y5UfhksjiZF3a4S6XDPI"",""spotify:artist:1gkSl4XpHIHI4I1WQbfXOE"",""Dick in the Air"",""spotify:album:5CH9B06MAZnydgiQ3dWxIl"",190240,""Rub""}, {1,""M.I.A."",""spotify:track:6iBaw0T7b3pTmMH1eMrul0"",""spotify:artist:0QJIPDAEDILuo8AIq3pMuU"",""Bad Girls"",""spotify:album:28GeLpi0VnbGYwLzZzfFUb"",227520,""Matangi""}, … {64,""Fiona Apple"",""spotify:track:6wntNrWu9bXBNPmujdwqhL"",""spotify:artist:3g2kUQ6tHLLbmkV7T4GPtL"",""Across the Universe"",""spotify:album:6zHpl6EFvM9Z7I9JGETeUz"",307333,""Pleasantville -Music From The Motion Picture""}]",10,15714900,61,null
"""BBG""","""true""",127,1493683200,26,25,1,"[{0,""Ed Sheeran"",""spotify:track:7qiZfU4dY1lWllzX7mPBI3"",""spotify:artist:6eUKZXaKkcviH0Ku9w2n3V"",""Shape of You"",""spotify:album:3T4tUhGYeRNVUGevb0wThu"",233712,""÷""}, {1,""The Chainsmokers"",""spotify:track:0QsvXIfqM0zZoerQfsI9lm"",""spotify:artist:69GGBxA162lTqCwzJG5jLp"",""Don't Let Me Down"",""spotify:album:3ShQFl9FladFKlonwPGZFc"",208053,""The Chainsmokers- Japan Special Edition""}, … {25,""Starley"",""spotify:track:78rIJddV4X0HkNAInEcYde"",""spotify:artist:02A3cEvlLLCbIMVDrK2GHV"",""Call On Me - Ryan Riback Extended Remix"",""spotify:album:4pZbAcipxqdFcwjiFkuwes"",222040,""Call On Me""}]",3,5549763,24,null
"""Killin' It""","""true""",132,1487116800,10,10,1,"[{0,""Misfits"",""spotify:track:3TQ4GxL3oghmLVpUD1oKmb"",""spotify:artist:1cXi8ALPQCBHZbf0EgP4Ey"",""We Are 138 - 1995 Digital Remaster"",""spotify:album:7dUrduNuWHYQ8pdDKtBedD"",101040,""Collection 2""}, {1,""The Chemical Brothers"",""spotify:track:4bz7uB4edifWKJXSDxwHcs"",""spotify:artist:1GhPHrq36VKCY3ucVaZCfo"",""Galvanize"",""spotify:album:715rku55lldqdw9WiEuPgi"",393813,""Push The Button""}, … {9,""Denzel Curry"",""spotify:track:6R0GRYk2vs2XuBVemYK5YZ"",""spotify:artist:6fxyWrfmjcbj5d12gXeiNV"",""Ultimate"",""spotify:album:5r0LVuUjx659ddoV15P5MI"",189974,""Ultimate""}]",9,2443305,10,null
…,…,…,…,…,…,…,…,…,…,…,…
"""Check it out!""","""true""",999841,1509148800,82,81,1,"[{0,""Belmont Lights"",""spotify:track:2o8v04yRJD85hsooabASfR"",""spotify:artist:2J40UTr83erJwVgpc0pMPC"",""Raise Up"",""spotify:album:3TlkE2IUAfEpycuv3YvePN"",212400,""Empyrean Kings""}, {1,""Sway Clarke"",""spotify:track:1NqYMFEyjN9UUwMMtJ82Fw"",""spotify:artist:1chXzMmrrsZU1vnIMMepTr"",""Secret Garden"",""spotify:album:1yQbICLXu1fuPgMnm8yCZV"",236831,""Secret Garden""}, … {81,""Chris Young"",""spotify:tr

In [11]:
df.sort_values(by='num_followers')

,name,collaborative,pid,modified_at,num_tracks,num_albums,num_followers,tracks,num_edits,duration_ms,num_artists
0,Throwbacks,false,0,1493424000,52,47,1,"[{'pos': 0, 'artist_name': 'Missy Elliott', 't...",6,11532414,37
1242,erika,false,999242,1489276800,10,10,1,"[{'pos': 0, 'artist_name': 'Tijuana Panthers',...",2,2119041,10
1241,xXx,false,999241,1508457600,98,96,1,"[{'pos': 0, 'artist_name': 'Bryant Myers', 'tr...",8,26822676,57
1240,Padre,false,999240,1508889600,17,17,1,"[{'pos': 0, 'artist_name': 'Steve Miller Band'...",11,4274638,14
1238,Yacht Rock,false,999238,1450137600,58,45,1,"[{'pos': 0, 'artist_name': 'The Doobie Brother...",2,14083060,33
...,...,...,...,...,...,...,...,...,...,...,...
1543,Electro Chill,false,999543,1509321600,186,162,17,"[{'pos': 0, 'artist_name': 'Forss', 'track_uri...",119,52947630,136
241,Tegan and Sara,false,241,1464998400,151,25,18,"[{'pos': 0, 'artist_name': 'Tegan and Sara', '...",17,29003627,7
1675,Sunny,true,999675,1438992000,51,49,18,"[{'pos': 0, 'artist_name': 'Los Lonely Boys', ...",30,11748191,37
255,Christmas Mix,false,255,1387152000,32,28,22,"[{'pos': 0, 'artist_name': 'Neko Case', 'track...",3,6772221,25


In [4]:
LstJson()[0]

TypeError: 'NoneType' object is not subscriptable

In [24]:
df.pid.unique()

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

In [20]:
df.head()

,name,collaborative,pid,modified_at,num_tracks,num_albums,num_followers,tracks,num_edits,duration_ms,num_artists
0,Throwbacks,false,0,1493424000,52,47,1,"[{'pos': 0, 'artist_name': 'Missy Elliott', 't...",6,11532414,37
1,Awesome Playlist,false,1,1506556800,39,23,1,"[{'pos': 0, 'artist_name': 'Survivor', 'track_...",5,11656470,21
2,korean,false,2,1505692800,64,51,1,"[{'pos': 0, 'artist_name': 'Hoody', 'track_uri...",18,14039958,31
3,mat,false,3,1501027200,126,107,1,"[{'pos': 0, 'artist_name': 'Camille Saint-Saën...",4,28926058,86
4,90s,false,4,1401667200,17,16,2,"[{'pos': 0, 'artist_name': 'The Smashing Pumpk...",7,4335282,16


In [17]:
playlist.keys()

dict_keys(['name', 'collaborative', 'pid', 'modified_at', 'num_tracks', 'num_albums', 'num_followers', 'tracks', 'num_edits', 'duration_ms', 'num_artists'])

# The Million Playlist Dataset
(Documentation updated Aug 5, 2020)

The Million Playlist Dataset contains 1,000,000 playlists created by
users on the Spotify platform.  It can be used by researchers interested
in exploring how to improve the music listening experience.

## What's in the Million Playlist Dataset (MPD)
The MPD contains a million user-generated playlists. These playlists
were created during the period of January 2010 through October 2017.
Each playlist in the MPD contains a playlist title, the track list
(including track metadata) editing information (last edit time, 
number of playlist edits) and other miscellaneous information 
about the playlist. See the **Detailed
Description** section for more details.

## License
Usage of the Million Playlist Dataset is subject to these 
[license terms](https://www.aicrowd.com/challenges/spotify-million-playlist-dataset-challenge/challenge_rules)

## Citing the Million Playlist Dataset
To use this dataset, please cite the following [paper](https://dl.acm.org/doi/abs/10.1145/3240323.3240342):

*Ching-Wei Chen, Paul Lamere, Markus Schedl, and Hamed Zamani. Recsys Challenge 2018: Automatic Music Playlist Continuation. In Proceedings of the 12th ACM Conference on Recommender Systems (RecSys ’18), 2018.*

## Getting the dataset
The dataset is available at [https://www.aicrowd.com/challenges/spotify-million-playlist-dataset-challenge](https://www.aicrowd.com/challenges/spotify-million-playlist-dataset-challenge)

## Verifying your dataset
You can validate the dataset by checking the md5 hashes of the data.  From the top level directory of the MPD:
   
    % md5sum -c md5sums
  
This should print out OK for each of the 1,000 slice files in the dataset.

You can also compute a number of statistics for the dataset as follows:

    % python src/stats.py data
  
The output of this program should match what is in 'stats.txt'. Depending on how 
fast your computer is, stats.py can take 30 minutes or more to run.

## Detailed description
The Million Playlist Dataset consists of 1,000 slice files. These files have the naming convention of:

mpd.slice._STARTING\_PLAYLIST\_ID\_-\_ENDING\_PLAYLIST\_ID_.json

For example, the first 1,000 playlists in the MPD are in a file called 
`mpd.slice.0-999.json` and the last 1,000 playlists are in a file called
`mpd.slice.999000-999999.json`.

Each slice file is a JSON dictionary with two fields:
*info* and *playlists*.

### `info` Field
The info field is a dictionary that contains general information about the particular slice:

   * **slice** - the range of slices that in in this particular file - such as 0-999
   * ***version*** -  - the current version of the MPD (which should be v1)
   * ***description*** - a description of the MPD
   * ***license*** - licensing info for the MPD
   * ***generated_on*** - a timestamp indicating when the slice was generated.

### `playlists` field 
This is an array that typically contains 1,000 playlists. Each playlist is a dictionary that contains the following fields:


* ***pid*** - integer - playlist id - the MPD ID of this playlist. This is an integer between 0 and 999,999.
* ***name*** - string - the name of the playlist 
* ***description*** - optional string - if present, the description given to the playlist.  Note that user-provided playlist descrptions are a relatively new feature of Spotify, so most playlists do not have descriptions.
* ***modified_at*** - seconds - timestamp (in seconds since the epoch) when this playlist was last updated. Times are rounded to midnight GMT of the date when the playlist was last updated.
* ***num_artists*** - the total number of unique artists for the tracks in the playlist.
* ***num_albums*** - the number of unique albums for the tracks in the playlist
* ***num_tracks*** - the number of tracks in the playlist
* ***num_followers*** - the number of followers this playlist had at the time the MPD was created. (Note that the follower count does not including the playlist creator)
* ***num_edits*** - the number of separate editing sessions. Tracks added in a two hour window are considered to be added in a single editing session.
* ***duration_ms*** - the total duration of all the tracks in the playlist (in milliseconds)
* ***collaborative*** -  boolean - if true, the playlist is a collaborative playlist. Multiple users may contribute tracks to a collaborative playlist.
* ***tracks*** - an array of information about each track in the playlist. Each element in the array is a dictionary with the following fields:
   * ***track_name*** - the name of the track
   * ***track_uri*** - the Spotify URI of the track
   * ***album_name*** - the name of the track's album
   * ***album_uri*** - the Spotify URI of the album
   * ***artist_name*** - the name of the track's primary artist
   * ***artist_uri*** - the Spotify URI of track's primary artist
   * ***duration_ms*** - the duration of the track in milliseconds
   * ***pos*** - the position of the track in the playlist (zero-based)

Here's an example of a typical playlist entry:
  
        {
            "name": "musical",
            "collaborative": "false",
            "pid": 5,
            "modified_at": 1493424000,
            "num_albums": 7,
            "num_tracks": 12,
            "num_followers": 1,
            "num_edits": 2,
            "duration_ms": 2657366,
            "num_artists": 6,
            "tracks": [
                {
                    "pos": 0,
                    "artist_name": "Degiheugi",
                    "track_uri": "spotify:track:7vqa3sDmtEaVJ2gcvxtRID",
                    "artist_uri": "spotify:artist:3V2paBXEoZIAhfZRJmo2jL",
                    "track_name": "Finalement",
                    "album_uri": "spotify:album:2KrRMJ9z7Xjoz1Az4O6UML",
                    "duration_ms": 166264,
                    "album_name": "Dancing Chords and Fireflies"
                },
                {
                    "pos": 1,
                    "artist_name": "Degiheugi",
                    "track_uri": "spotify:track:23EOmJivOZ88WJPUbIPjh6",
                    "artist_uri": "spotify:artist:3V2paBXEoZIAhfZRJmo2jL",
                    "track_name": "Betty",
                    "album_uri": "spotify:album:3lUSlvjUoHNA8IkNTqURqd",
                    "duration_ms": 235534,
                    "album_name": "Endless Smile"
                },
                {
                    "pos": 2,
                    "artist_name": "Degiheugi",
                    "track_uri": "spotify:track:1vaffTCJxkyqeJY7zF9a55",
                    "artist_uri": "spotify:artist:3V2paBXEoZIAhfZRJmo2jL",
                    "track_name": "Some Beat in My Head",
                    "album_uri": "spotify:album:2KrRMJ9z7Xjoz1Az4O6UML",
                    "duration_ms": 268050,
                    "album_name": "Dancing Chords and Fireflies"
                },
                // 8 tracks omitted
                {
                    "pos": 11,
                    "artist_name": "Mo' Horizons",
                    "track_uri": "spotify:track:7iwx00eBzeSSSy6xfESyWN",
                    "artist_uri": "spotify:artist:3tuX54dqgS8LsGUvNzgrpP",
                    "track_name": "Fever 99\u00b0",
                    "album_uri": "spotify:album:2Fg1t2tyOSGWkVYHlFfXVf",
                    "duration_ms": 364320,
                    "album_name": "Come Touch The Sun"
                }
            ],

        }


## Tools
There are some tools in the src/ directory that you can use with the dataset:
+ check.py - checks to se that the MPD is correct
+ deeper_stats.py - shows deep stats for the MPD
+ descriptions.py - surfaces the most common descriptions in the MPD
+ print.py - prints the full MPD
+ show.py - shows playlists by id or id range in the MPD
+ stats.py - iterates over the million playlist dataset and outputs info about what is in there.

## How was the dataset built
The Million Playist Dataset is created by sampling playlists from the billions of playlists that Spotify users have created over the years.  Playlists that meet the following criteria are selected at random:

 * Created by a user that resides in the United States and is at least 13 years old
 * Was a public playlist at the time the MPD was generated
 * Contains at least 5 tracks
 * Contains no more than 250 tracks
 * Contains at least 3 unique artists
 * Contains at least 2 unique albums
 * Has no local tracks (local tracks are non-Spotify tracks that a user has on their local device)
 * Has at least one follower (not including the creator)
 * Was created after January 1, 2010 and before December 1, 2017
 * Does not have an offensive title
 * Does not have an adult-oriented title if the playlist was created by a user under 18 years of age

Additionally, some playlists have been modified as follows:

 * Potentially offensive playlist descriptions are removed
 * Tracks added on or after November 1, 2017 are removed

Playlists are sampled randomly, for the most part, but with some dithering to disguise the true distribution of playlists within Spotify. [Paper tracks](https://en.wikipedia.org/wiki/Fictitious_entry) may be added to some playlists to help us identify improper/unlicensed use of the dataset.

## Overall demographics of users contributing to the MPD

### Gender
 * Male: 45%
 * Female: 54%
 * Unspecified: 0.5%
 * Nonbinary: 0.5%

### Age
 * 13-17:  10%
 * 18-24:  43%
 * 25-34:  31%
 * 35-44:   9%
 * 45-54:   4%
 * 55+:     3%

### Country
 * US: 100%


## Who built the dataset
The million playlist dataset was built by the following researchers @ Spotify:

* Cedric De Boom
* Paul Lamere
* Ching-Wei Chen
* Ben Carterette
* Christophe Charbuillet
* Jean Garcia-Gathright
* James Kirk
* James McInerney
* Vidhya Murali
* Hugh Rawlinson
* Sravana Reddy
* Marc Romejin
* Romain Yon
* Yu Zhao

In [130]:
process_playlists('/home/iceberg/notebooks/json')

===== 0 ====
name:           Throwbacks
last_modified:  2017-04-29 00:00:00
num edits:  6
num followers:  1
num artists:  37
num albums:  47
num tracks:  52

     1 Lose Control (feat. Ciara & Fat Man Scoop) - The Cookbook - Missy Elliott
     2 Toxic - In The Zone - Britney Spears
     3 Crazy In Love - Dangerously In Love (Alben für die Ewigkeit) - Beyoncé
     4 Rock Your Body - Justified - Justin Timberlake
     5 It Wasn't Me - Hot Shot - Shaggy
     6 Yeah! - Confessions - Usher
     7 My Boo - Confessions - Usher
     8 Buttons - PCD - The Pussycat Dolls
     9 Say My Name - The Writing's On The Wall - Destiny's Child
    10 Hey Ya! - Radio Mix / Club Mix - Speakerboxxx/The Love Below - OutKast
    11 Promiscuous - Loose - Nelly Furtado
    12 Right Where You Want Me - Radio Edit Version - Right Where You Want Me - Jesse McCartney
    13 Beautiful Soul - Beautiful Soul - Jesse McCartney
    14 Leavin' - Departure - Recharged - Jesse McCartney
    15 Me & U - Cassie - Cassie
    

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [49]:
pd.read_json('./testfile.json', )

ValueError: Mixing dicts with non-Series may lead to ambiguous ordering.

In [48]:
df

DataFrame[_corrupt_record: string]